In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
loss = nn.BCEWithLogitsLoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(input, target)
output.backward()

In [ ]:
import sys
sys.path.append("../")
from ops import sigmoid_cross_entropy_with_logits, reduce_mean

In [ ]:
D_logits = torch.tensor(np.random.rand(10))
onelike = torch.ones_like(D_logits)

x = sigmoid_cross_entropy_with_logits(D_logits, onelike)
print(x, reduce_mean(x))

In [ ]:
X_prev = np.load("../dataset/not_augmented/X_tr.npy")

In [ ]:
X_prev.shape

In [ ]:
import os

os.path.join("./abc", "/xyz", "name")

In [ ]:
import os
import numpy as np
from pypianoroll import Multitrack, Track 
import pypianoroll as pypiano
import torch
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import ipdb
from main import creat_directory
import pypianoroll

def find_pitch(song,volume=40):   # song shape(128,128), which is (time step, pitch)
    for time in range(song.shape[0]):
        step = song[time,:]
        max_index = np.argmax(step)
        for i in range(len(step)):
            if i ==max_index:
                song[time,i] = volume
            else:
                song[time,i] = 0
    return song

def reshape_bar(song):
    eight_bar = song[0]
    for i in range(7):
        b = song[i+1]
        eight_bar  = np.concatenate((eight_bar,b),axis=0)
    eight_bar = eight_bar.astype(float)
    print("A bar's shape: {}".format(eight_bar.shape))
    return eight_bar

def make_a_track(eight_bar_binarized,track_name ='melody' ,instrument=0):
    track = pypiano.StandardTrack(pianoroll=eight_bar_binarized, program=instrument, is_drum=False,name=track_name)
    return track

def make_a_demo(track1,track2,song_idx):
    sample_name = 'sample_'+str(song_idx)
    tempo = np.full(768, 120, dtype=float)
    multitrack = Multitrack(tracks=[track1,track2], tempo=tempo,resolution=4)
    multitrack.plot()
    plt.savefig('./output_demo/'+sample_name+'.png')
    return multitrack


def chord_list(chord,idx):

    one_song_chord = chord[idx]
    song_chord = []
    for i in range(len(one_song_chord)):
        bar_idx = []
        one_bar_chord = one_song_chord[i]
        bar_idx.append(int(one_bar_chord[0][12]))
        max_idx = np.argmax(one_bar_chord[:11])
        bar_idx.append(max_idx)
        song_chord.append(bar_idx)
    return song_chord


def build_chord_map():
    c_maj  = [60,64,67]
    c_min  = [60,63,67]
    chord_map = []
    chord_list_maj = []
    chord_list_min = []
    chord_list_maj.append(c_maj)
    chord_list_min.append(c_min)
    for i in range(11):
        chord = [x+1 for x in c_maj] 
        c_maj = chord
        chord_list_maj.append(chord)
        chord_min = [x+1 for x in c_min]
        chord_list_min.append(chord_min)
        c_min = chord_min
    chord_map.append(chord_list_maj)
    chord_list_min[:] = chord_list_min[9:] + chord_list_min[0:9]
    chord_map.append(chord_list_min)
    return chord_map

def decode_chord(maj_min,which_chord):

    chord_map = build_chord_map()
    chord = chord_map[maj_min][which_chord]

    return chord

def get_chord(song_chord):
    chord_player = []
    for item in song_chord:
        maj_min = item[0]
        which_chord = item[1]
        answer_chord = decode_chord(maj_min,which_chord)
        chord_player.append(answer_chord)
    return chord_player

def make_chord_track(chord,instrument,volume=40):
    pianoroll = np.zeros((128, 128))
    for i in range(len(chord)):
        st = 16*i
        ed = st + 16
        chord_pitch = chord[i]
        pianoroll[st:ed, chord_pitch] = volume
    track = pypiano.StandardTrack(pianoroll=pianoroll, program=instrument, is_drum=False,
                  name='chord')
    return track

In [ ]:
model_ver = int(input("Model: "))
attempt = int(input("Attempt: "))
init_path = f"./{model_ver}/{model_ver}.{attempt}"
output_path = init_path + "/output"
data = np.load(os.path.join(output_path, 'output_songs.npy'))
chord = np.load(os.path.join(output_path, 'output_chords.npy'))
# instrument = input('which instrument you want to play? from 0 to 128,default=0:')
# volume     = input('how loud you want to play? from 1 to 127,default= 40:')
instrument = 16
volume = 40
sample_name = "test_song"
for i in range(5):
    # if i % 100 == 0:
    one_song = data[i]
    song = []
    for item in one_song:
        item = item.reshape(16,128)
        song.append(item)

    eight_bar = reshape_bar(song)
    eight_bar_binarized = find_pitch(eight_bar,volume)
    track = make_a_track(eight_bar_binarized,instrument)
    
    save_path = init_path + "/demo"
    creat_directory(save_path)
    song_chord = chord_list(chord,i)
    chord_player = get_chord(song_chord)
    np.save(os.path.join(save_path, f'chord_{i}.npy'),chord_player)
    chord_track = make_chord_track(chord_player,instrument,volume)
    multitrack = make_a_demo(track,chord_track,i)
    # pypianoroll.write(path="test", multitrack=multitrack)
    print(str(sample_name)+'saved')
    print(multitrack)

In [ ]:
from midi2audio import FluidSynth


In [ ]:
fs = FluidSynth()


In [ ]:
fs.midi_to_audio('./3/3.2/demo/test_song_3.mid', './3/3.2/demo/test_3.wav')

In [18]:
import subprocess

sample_rate = 44100

def midi_to_mp3(midi_file, audio_file, sound_font="/users/eleves-b/2021/minh.pham/.fluidsynth/Abbey-Steinway-D-v1.9.sf2"):
    # Convert modified MIDI to WAV using FluidSynth
    subprocess.call(["/users/eleves-b/2021/minh.pham/fluidsynth-2.3.2/build/src/fluidsynth", '-ni', sound_font, midi_file, '-F', audio_file, '-r', str(sample_rate)])

    # Convert WAV to MP3 using FFmpeg
    # subprocess.call(['ffmpeg', '-i', 'output.wav', '-codec:a', 'libmp3lame', output_file])

    # Clean up the intermediate WAV file
    # subprocess.call(['rm', 'output.wav'])

In [19]:
midi_to_mp3('./3/3.2/demo/test_song_3.mid', './3/3.2/demo/test_3.wav')

fluidsynth: panic: An error occurred while reading from stdin.
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: panic: An error occurred while reading from stdin.
fluidsynth: warning: Instrument not found on channel 0 [bank=0 prog=40], substituted [bank=0 prog=0]
fluidsynth: warning: Instrument not found on channel 1 [bank=0 prog=40], substituted [bank=0 prog=0]


FluidSynth runtime version 2.3.2
Copyright (C) 2000-2023 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file './3/3.2/demo/test_3.wav'..
